In [28]:
import pandas as pd
import requests
import os
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
import hvplot.pandas

In [29]:
top_5_mentions_stocks = requests.get('https://apewisdom.io/api/v1.0/filter/wallstreetbets').json()['results'][:5]

In [30]:
top_5_mentions_stocks

[{'rank': 1,
  'ticker': 'SPY',
  'name': 'SPDR S&amp;P 500 ETF Trust',
  'mentions': '187',
  'upvotes': '619',
  'rank_24h_ago': '1',
  'mentions_24h_ago': '780'},
 {'rank': 2,
  'ticker': 'GME',
  'name': 'GameStop',
  'mentions': '112',
  'upvotes': '690',
  'rank_24h_ago': '2',
  'mentions_24h_ago': '380'},
 {'rank': 3,
  'ticker': 'TSLA',
  'name': 'Tesla',
  'mentions': '63',
  'upvotes': '168',
  'rank_24h_ago': '3',
  'mentions_24h_ago': '346'},
 {'rank': 4,
  'ticker': 'AMC',
  'name': 'AMC Entertainment',
  'mentions': '60',
  'upvotes': '261',
  'rank_24h_ago': '8',
  'mentions_24h_ago': '109'},
 {'rank': 5,
  'ticker': 'AMD',
  'name': 'AMD',
  'mentions': '39',
  'upvotes': '147',
  'rank_24h_ago': '11',
  'mentions_24h_ago': '80'}]

In [36]:
top_5_mentions_stocks_df = pd.DataFrame.from_records(top_5_mentions_stocks)
top_5_mentions_stocks_df

,rank,ticker,name,mentions,upvotes,rank_24h_ago,mentions_24h_ago
0,1,SPY,SPDR S&amp;P 500 ETF Trust,187,619,1,780
1,2,GME,GameStop,112,690,2,380
2,3,TSLA,Tesla,63,168,3,346
3,4,AMC,AMC Entertainment,60,261,8,109
4,5,AMD,AMD,39,147,11,80


In [37]:
top_5_mentions_stocks_df.set_index('ticker', inplace=True)

In [38]:
top_5_mentions_stocks_df

,rank,name,mentions,upvotes,rank_24h_ago,mentions_24h_ago
ticker,,,,,,
SPY,1,SPDR S&amp;P 500 ETF Trust,187,619,1,780
GME,2,GameStop,112,690,2,380
TSLA,3,Tesla,63,168,3,346
AMC,4,AMC Entertainment,60,261,8,109
AMD,5,AMD,39,147,11,80


In [39]:
top_5_mentions_stocks_df=pd.DataFrame.from_dict(top_5_mentions_stocks_df, orient='columns')
top_5_mentions_stocks_df

,rank,name,mentions,upvotes,rank_24h_ago,mentions_24h_ago
ticker,,,,,,
SPY,1,SPDR S&amp;P 500 ETF Trust,187,619,1,780
GME,2,GameStop,112,690,2,380
TSLA,3,Tesla,63,168,3,346
AMC,4,AMC Entertainment,60,261,8,109
AMD,5,AMD,39,147,11,80


In [40]:
top_5_mentions_stocks_df.drop(columns=['rank','name','upvotes','rank_24h_ago','mentions_24h_ago'],inplace=True)                              

In [43]:
top_5_mentions_stocks_df

,mentions
ticker,
SPY,187
GME,112
TSLA,63
AMC,60
AMD,39


In [44]:
type(top_5_mentions_stocks_df['mentions'].iloc[0])

str

In [45]:
top_5_mentions_stocks_df = pd.to_numeric(top_5_mentions_stocks_df['mentions'], errors='coerce') #ya!!

In [46]:
top_5_mentions_stocks_df.hvplot(kind="bar", title='Mentions in last 24 hours for top 5 stocks')

:Bars   [ticker]   (mentions)

In [65]:
top_stocks_tickers = []

for stock in top_5_mentions_stocks:
    top_stocks_tickers.append(stock['ticker'])
top_stocks_tickers

['SPY', 'GME', 'TSLA', 'AMC', 'AMD']

In [66]:
load_dotenv('keys.env')

True

In [67]:
alpaca_api_key = os.getenv('ALPACA_API_KEY')
alpaca_secret_key = os.getenv('ALPACA_SECRET_KEY')
type(alpaca_api_key)

str

In [68]:
alpaca = tradeapi.REST(alpaca_api_key, alpaca_secret_key, api_version = 'v2')

start_date = pd.Timestamp('2022-05-13', tz='America/New_York').isoformat()
# end_date = pd.Timestamp('2022-05-13', tz='America/New_York').isoformat() 
prices_df = alpaca.get_bars(top_stocks_tickers, timeframe = '15Min', start = start_date).df
# , end = end_date
spy = prices_df[prices_df['symbol'] == 'SPY'].drop('symbol', axis = 1)
gme = prices_df[prices_df['symbol'] == 'GME'].drop('symbol', axis = 1)
tsla = prices_df[prices_df['symbol'] == 'TSLA'].drop('symbol', axis = 1)
amc = prices_df[prices_df['symbol'] == 'AMC'].drop('symbol', axis = 1)
amd = prices_df[prices_df['symbol'] == 'AMD'].drop('symbol', axis = 1)

prices_df = pd.concat([spy['close'], aapl['close'], tsla['close'], gme['close'], qqq['close']], axis = 1, keys = ['SPY', 'GME', 'TSLA', 'AMC', 'AMD'])
prices_df

,SPY,GME,TSLA,AMC,AMD
timestamp,,,,,
2022-05-13 08:00:00+00:00,396.48,93.99,756.44,93.99,89.07
2022-05-13 08:15:00+00:00,396.99,94.50,756.20,94.50,89.23
2022-05-13 08:30:00+00:00,396.59,94.30,754.75,94.30,89.00
2022-05-13 08:45:00+00:00,396.51,94.00,755.00,94.00,88.92
2022-05-13 09:00:00+00:00,397.02,94.49,758.69,94.49,89.10
...,...,...,...,...,...
2022-05-13 22:45:00+00:00,401.61,NaN,773.10,NaN,95.10
2022-05-13 23:00:00+00:00,401.72,98.00,773.50,98.00,95.20
2022-05-13 23:15:00+00:00,401.76,98.00,774.49,98.00,95.25


In [69]:
# prices_df_plot = (prices_df.pct_change()).hvplot()
prices_df_plot = (1 + prices_df.pct_change()).cumprod().hvplot()
prices_df_plot

:NdOverlay   [Variable]
   :Curve   [timestamp]   (value)

In [70]:
# top_5_stocks_prices = {'mentions': int(top_5_mentions_stocks[0]['mentions']), 'mentions_24h_ago': int(top_5_mentions_stocks[0]['mentions_24h_ago'])}
# top_5_stocks_prices

In [71]:
mentions_top_5_plot = top_5_mentions_stocks_df.hvplot(kind='bar')
mentions_top_5_plot

:Bars   [ticker]   (mentions)

In [72]:
overlayplot = prices_df_plot + mentions_top_5_plot
# hvplot - google 2 plot same graph

In [73]:
overlayplot

:Layout
   .NdOverlay.I   :NdOverlay   [Variable]
      :Curve   [timestamp]   (value)
   .Bars.Mentions :Bars   [ticker]   (mentions)